### Madina Hayva Putri - 5220411187
### Pemrosesan Teks Teori - B

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [34]:
data_manual = pd.read_csv("/content/data_manual.csv")
data_self_training = pd.read_csv("/content/data_self_training.csv")

In [35]:
print("data_manual:\n")
data_manual.head()

data_manual:



,komentar,final_cleaned,label
0,knp aga gelap ka,gelap,neutral
1,"kaa masukan nih yaa\naudionya benahin ka, enta...",masuk audio benahin mick suara kakanya kedenge...,positive
2,Kak plis membeli mainan ❤🎉,tolong beli main,neutral
3,Ci bikin konten make up lagi ci,bikin konten make,neutral
4,KK konten sama jio dong,konten,neutral


In [36]:
print("\ndata_self_training: \n")
data_self_training.head()


data_self_training: 



,komentar,final_cleaned
0,Kalau berani review nastar saya Umma Delight n...,berani review nastar delight nama tasyi
1,Itu mochi China mungkin ada yang jualan di Ind...,mochi china jual indonesia cina pakai kelapa c...
2,Ke GBK kak ada pucuk culinary festival loh cum...,pucuk culinary festival
3,Ini dinda sih pemanisnya ❤,dinda
4,"Suami orang Hakka ya ci, mertua masaknya Pak C...",suami orang haka mertua masak cham bakmi goren...


## Vektorisasi TF-IDF

In [37]:
X_manual = data_manual["final_cleaned"]
y_manual = data_manual["label"]

vectorizer_manual = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),
    stop_words=None
)

X_manual_tfidf = vectorizer_manual.fit_transform(X_manual)

print("Ukuran X_manual_tfidf:", X_manual_tfidf.shape)

Ukuran X_manual_tfidf: (301, 2674)


## Melatih Model

In [38]:
model_self_training = LogisticRegression(max_iter=2000)
model_self_training.fit(X_manual_tfidf, y_manual)

print("Model telah berhasil dilatih!")

Model telah berhasil dilatih!


In [39]:
X_self_training = data_self_training["final_cleaned"]
X_self_training_tfidf = vectorizer_manual.transform(X_self_training)

print("Ukuran X_self_training_tfidf:", X_self_training_tfidf.shape)

Ukuran X_self_training_tfidf: (700, 2674)


### Memprediksi Data

In [40]:
y_pred_self_training = model_self_training.predict(X_self_training_tfidf)

print("Jumlah prediksi yang dihasilkan:", len(y_pred_self_training))
data_self_training['label_manual'] = y_pred_self_training
data_self_training.head()

Jumlah prediksi yang dihasilkan: 700


,komentar,final_cleaned,label_manual
0,Kalau berani review nastar saya Umma Delight n...,berani review nastar delight nama tasyi,positive
1,Itu mochi China mungkin ada yang jualan di Ind...,mochi china jual indonesia cina pakai kelapa c...,neutral
2,Ke GBK kak ada pucuk culinary festival loh cum...,pucuk culinary festival,neutral
3,Ini dinda sih pemanisnya ❤,dinda,neutral
4,"Suami orang Hakka ya ci, mertua masaknya Pak C...",suami orang haka mertua masak cham bakmi goren...,neutral


## Model LLM

In [41]:
!pip install transformers accelerate sentencepiece -q

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

In [42]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


In [43]:
def classify_llm_batch(texts, batch_size=16):
    all_predictions = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        prompts = [f"""
Tentukan sentimen komentar berikut: positive, negative, atau neutral.
Jawab 1 kata saja.

Komentar: "{text}"
Jawaban:
""" for text in batch_texts]

        inputs = tokenizer(prompts, return_tensors="pt", padding=True, padding_side="left", truncation=True).to(model.device)

        outputs = model.generate(**inputs, max_new_tokens=10, do_sample=False, temperature=0.0)

        generated_tokens = outputs[:, inputs.input_ids.shape[1]:]

        for j in range(len(batch_texts)):
            result = tokenizer.decode(generated_tokens[j], skip_special_tokens=True).lower().strip()
            if "positive" in result and "negative" not in result and "neutral" not in result:
                all_predictions.append("positive")
            elif "negative" in result and "positive" not in result and "neutral" not in result:
                all_predictions.append("negative")
            elif "neutral" in result and "positive" not in result and "negative" not in result:
                all_predictions.append("neutral")
            else:
                if "positive" in result:
                    all_predictions.append("positive")
                elif "negative" in result:
                    all_predictions.append("negative")
                elif "neutral" in result:
                    all_predictions.append("neutral")
                else:
                    all_predictions.append("neutral")
    return all_predictions

print("Batch classification function for LLM created.")

Batch classification function for LLM created.


In [44]:
cleaned_texts = data_self_training['final_cleaned'].fillna('').tolist()

y_pred_llm_batch = classify_llm_batch(cleaned_texts, batch_size=32)

data_self_training['label_llm'] = y_pred_llm_batch
data_self_training.head()

,komentar,final_cleaned,label_manual,label_llm
0,Kalau berani review nastar saya Umma Delight n...,berani review nastar delight nama tasyi,positive,positive
1,Itu mochi China mungkin ada yang jualan di Ind...,mochi china jual indonesia cina pakai kelapa c...,neutral,positive
2,Ke GBK kak ada pucuk culinary festival loh cum...,pucuk culinary festival,neutral,positive
3,Ini dinda sih pemanisnya ❤,dinda,neutral,neutral
4,"Suami orang Hakka ya ci, mertua masaknya Pak C...",suami orang haka mertua masak cham bakmi goren...,neutral,positive


In [45]:
data_self_training.to_csv("data_self_training_with_prediction.csv", index=False)